# Tempogram widget

Input from Novelty (or set `NOVELTY_FOR_TEMPOGRAM` below). By default runs novelty with default params to get input. Preceding params = re-run **novelty_widget.ipynb** or set override. This widget's params = live or Recompute.

In [1]:
import sys
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, Javascript, display

widget_dir = Path.cwd()
for candidate in [Path.cwd(), Path.cwd() / "widget", Path.cwd() / "scratch" / "widget", Path.cwd() / "notebooks" / "scratch" / "widget"]:
    if (candidate / "temporal_widget_helpers.py").exists():
        widget_dir = candidate
        break
sys.path.insert(0, str(widget_dir))
project_root = widget_dir
for _ in range(6):
    if (project_root / "src" / "dijon").exists():
        break
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

from temporal_widget_helpers import (
    build_audio_with_waveform_html,
    NOVELTY_DEFAULTS,
    WINDOW_SEC_OPTIONS,
    HOP_SEC_OPTIONS,
    eager_compute_tempogram,
    get_novelty_for_method,
    get_tempogram_for_methods,
    load_audio_and_markers,
    novelty_params_key,
    tempo_from_tempogram,
    zoom_level_to_half_sec,
)

%matplotlib inline

In [2]:
TRACK = "YTB-005"
MARKER = ""

x, sr, seg_start, seg_end, marker_label = load_audio_and_markers(TRACK, MARKER or None)
duration_s = len(x) / sr
print(f"Track: {TRACK} | Marker: {marker_label} | Duration: {duration_s:.2f}s @ {sr} Hz")

Track: YTB-005 | Marker: FULL | Duration: 175.48s @ 22050 Hz


In [3]:
novelty_cache = {}
tempogram_cache = {}
print("Caches initialized. Default upstream novelty + eager tempogram run in next cell.")

Caches initialized. Default upstream novelty + eager tempogram run in next cell.


**Override (optional)** Set `NOVELTY_FOR_TEMPOGRAM` to a dict with keys `nov_100`, `nm`, `novelty_key` to use custom novelty output. Leave `None` to use default novelty below.

In [4]:
def _is_empty(v):
    return v is None or v == "" or (isinstance(v, dict) and len(v) == 0)

NOVELTY_FOR_TEMPOGRAM = None

In [5]:
# Default upstream novelty is computed once here (unless override), then tempogram is eagerly computed once.
nm_default = "spectral"
d = NOVELTY_DEFAULTS[nm_default]
N, H, gamma, M = d["N"], d["H"], d.get("gamma") or 100.0, d.get("M", 0)
nov, nov_100 = get_novelty_for_method(novelty_cache, x, sr, nm_default, N, H, gamma, M)
nov_key = novelty_params_key(nm_default, N, H, gamma, M)
_nov_state = {"nov_100": nov_100, "nm": nm_default, "novelty_key": nov_key}

_nov_src = _nov_state if _is_empty(NOVELTY_FOR_TEMPOGRAM) else NOVELTY_FOR_TEMPOGRAM
nm_src = _nov_src.get("nm", "spectral")
d_src = NOVELTY_DEFAULTS[nm_src]
nov_key_src = _nov_src.get("novelty_key") or novelty_params_key(
    nm_src,
    d_src["N"],
    d_src["H"],
    d_src.get("gamma") or 100.0,
    d_src.get("M", 0),
)
_temp_state, tempogram_cache, temp_init_time, _ = eager_compute_tempogram(
    _nov_src["nov_100"],
    nov_key_src,
    tempogram_method="fourier",
    window_sec=5.0,
    hop_sec=0.1,
)
print(f"Eager tempogram init: {temp_init_time:.2f}s | Tempo: {_temp_state['tempo_bpm']:.1f} BPM")

Eager tempogram init: 0.92s | Tempo: 244.0 BPM


In [6]:
_temp_compact = dict(layout=widgets.Layout(margin="0 2px 2px 2px"))

temp_w_method = widgets.Dropdown(options=["fourier", "autocorr", "cyclic"], value="fourier", description="Tempogram:", **_temp_compact)
temp_w_window = widgets.Dropdown(options=WINDOW_SEC_OPTIONS, value=5.0, description="", **_temp_compact)
temp_w_hop_sec = widgets.Dropdown(options=HOP_SEC_OPTIONS, value=0.1, description="", **_temp_compact)
temp_w_zoom_enabled = widgets.Checkbox(value=False, description="Zoom on playhead", **_temp_compact)
temp_w_zoom_level = widgets.IntSlider(value=1, min=1, max=10, step=1, description="", **_temp_compact)

def _temp_label(txt):
    return widgets.HTML(f'<div style="font-size:9px;margin-top:2px;margin-bottom:0;line-height:1.2;color:#1565c0">{txt}</div>')

TEMP_FACTORY = {"tempogram_method": "fourier", "window_sec": 5.0, "hop_sec": 0.1, "zoom_enabled": False, "zoom_level": 1}
temp_defaults = {
    "tempogram_method": widgets.Dropdown(options=["fourier", "autocorr", "cyclic"], value=TEMP_FACTORY["tempogram_method"], **_temp_compact),
    "window_sec": widgets.Dropdown(options=WINDOW_SEC_OPTIONS, value=TEMP_FACTORY["window_sec"], **_temp_compact),
    "hop_sec": widgets.Dropdown(options=HOP_SEC_OPTIONS, value=TEMP_FACTORY["hop_sec"], **_temp_compact),
    "zoom_enabled": widgets.Checkbox(value=TEMP_FACTORY["zoom_enabled"], **_temp_compact),
    "zoom_level": widgets.IntSlider(value=TEMP_FACTORY["zoom_level"], min=1, max=10, **_temp_compact),
}

# Keep eager state from cell 6 if present; otherwise widget starts empty until Recompute
_temp_state = _temp_state if isinstance(_temp_state, dict) and "tempo_bpm" in _temp_state else {}

def temp_w_do_recompute(_):
    _nov_src = _nov_state if _is_empty(NOVELTY_FOR_TEMPOGRAM) else NOVELTY_FOR_TEMPOGRAM
    if not _nov_src:
        with temp_w_status_out:
            temp_w_status_out.clear_output()
            print("Run default-novelty cell above or set NOVELTY_FOR_TEMPOGRAM.")
        return
    import time
    t0 = time.perf_counter()
    tm = temp_w_method.value
    nm = _nov_src.get("nm", "spectral")
    N = NOVELTY_DEFAULTS[nm]["N"]
    d = NOVELTY_DEFAULTS[nm]
    nov_key = _nov_src.get("novelty_key") or novelty_params_key(nm, N, d["H"], d.get("gamma") or 100.0, d.get("M", 0))
    tempogram, T_coef, F_coef, aux = get_tempogram_for_methods(
        tempogram_cache, _nov_src["nov_100"], nov_key, tm,
        window_sec=temp_w_window.value, hop_sec=temp_w_hop_sec.value,
    )
    tempo_bpm = tempo_from_tempogram(tempogram, aux["F_coef_BPM"], tm, aux=aux)
    t_compute = time.perf_counter() - t0
    _temp_state.update({"tempogram": tempogram, "T_coef": T_coef, "F_coef": F_coef, "aux": aux, "tm": tm, "tempo_bpm": tempo_bpm})
    temp_w_redraw(0)
    with temp_w_status_out:
        temp_w_status_out.clear_output()
        print(f"Tempogram: {t_compute:.3f}s | Tempo: {tempo_bpm:.1f} BPM")

def temp_w_redraw(center_time=None, update_audio=True):
    if not _temp_state:
        return
    half = zoom_level_to_half_sec(temp_w_zoom_level.value, duration_s) if temp_w_zoom_enabled.value else duration_s
    xlo = max(0, (center_time or 0) - half) if temp_w_zoom_enabled.value and center_time is not None else 0
    xhi = min(duration_s, (center_time or 0) + half) if temp_w_zoom_enabled.value and center_time is not None else duration_s
    if not temp_w_zoom_enabled.value:
        xlo, xhi = 0, duration_s
    with temp_w_plot_out:
        temp_w_plot_out.clear_output(wait=True)
        T_coef, F_coef = _temp_state["T_coef"], _temp_state["F_coef"]
        extent = [T_coef[0], T_coef[-1], F_coef[0], F_coef[-1]]
        fig, ax = plt.subplots(1, 1, figsize=(12, 3))
        ax.imshow(_temp_state["tempogram"], aspect="auto", origin="lower", extent=extent, cmap="gray_r")
        ax.set_ylabel("Tempo (BPM)" if _temp_state["tm"] != "cyclic" else "Scale")
        ax.set_title(f"Tempogram ({_temp_state['tm']}) | Tempo: {_temp_state['tempo_bpm']:.1f} BPM")
        ax.set_xlim(xlo, xhi)
        ax.set_xlabel("Time (s)")
        plt.tight_layout()
        plt.show()
    if update_audio:
        with temp_w_audio_out:
            temp_w_audio_out.clear_output(wait=True)
            h = build_audio_with_waveform_html(x, x, sr, max_duration_sec=max(60, duration_s))
            display(HTML('<div id="temp-w-audio-container">' + h + '</div>'))
            display(Javascript("(function(){ var c = document.getElementById('temp-w-audio-container'); var a = c ? c.querySelector('audio') : null; if (!a) return; var last = -1; a.addEventListener('timeupdate', function(){ var t = a.currentTime; if (Math.abs(t - last) < 0.25) return; last = t; if (typeof Jupyter !== 'undefined' && Jupyter.notebook && Jupyter.notebook.kernel) Jupyter.notebook.kernel.execute('temp_w_zoom_from_playhead(' + t + ')'); }); })();"))
def temp_w_zoom_from_playhead(t):
    if temp_w_zoom_enabled.value and t is not None:
        temp_w_redraw(float(t), update_audio=False)

temp_w_zoom_enabled.observe(lambda _: temp_w_redraw(0) if _temp_state else None, names="value")
temp_w_zoom_level.observe(lambda _: temp_w_redraw(0) if _temp_state else None, names="value")
temp_w_method.observe(lambda _: temp_w_do_recompute(None), names="value")
temp_w_window.observe(lambda _: temp_w_do_recompute(None), names="value")
temp_w_hop_sec.observe(lambda _: temp_w_do_recompute(None), names="value")

_temp_param_widgets = {"tempogram_method": temp_w_method, "window_sec": temp_w_window, "hop_sec": temp_w_hop_sec, "zoom_enabled": temp_w_zoom_enabled, "zoom_level": temp_w_zoom_level}
def temp_w_reset(_):
    for k, w in temp_defaults.items():
        if k in ("zoom_enabled", "zoom_level"): continue
        try:
            v = w.value
            if v is None: continue
            if k in ("window_sec", "hop_sec"): v = float(v)
            elif k == "zoom_level": v = int(v)
            _temp_param_widgets[k].value = v
        except (TypeError, ValueError): pass
    temp_w_do_recompute(None)
def temp_w_set_default(_):
    for k in temp_defaults:
        temp_defaults[k].value = _temp_param_widgets[k].value
def temp_w_factory(_):
    for k, w in temp_defaults.items(): w.value = TEMP_FACTORY[k]
    temp_w_reset(None)

temp_w_recompute_btn = widgets.Button(description="Recompute", button_style="primary")
temp_w_recompute_btn.on_click(temp_w_do_recompute)
temp_w_status_out = widgets.Output()
temp_w_plot_out = widgets.Output()
temp_w_audio_out = widgets.Output()

temp_w_controls = widgets.VBox([
    widgets.HTML("<b>Tempogram</b>"), temp_w_method, _temp_label("Window (s)"), temp_w_window, _temp_label("Hop (s)"), temp_w_hop_sec,
    widgets.HTML("<b>Zoom</b>"), temp_w_zoom_enabled, _temp_label("Zoom (1=out, 10=in)"), temp_w_zoom_level,
    temp_w_recompute_btn,
    widgets.HTML("<b>Defaults</b>"), _temp_label("Tempogram"), temp_defaults["tempogram_method"], _temp_label("Window (s)"), temp_defaults["window_sec"], _temp_label("Hop (s)"), temp_defaults["hop_sec"],
    _temp_label("Zoom on"), temp_defaults["zoom_enabled"], _temp_label("Zoom level"), temp_defaults["zoom_level"],
    widgets.HBox([widgets.Button(description="RESET"), widgets.Button(description="SET TO DEFAULT")]), widgets.Button(description="Reset to factory", button_style="warning"),
], layout=widgets.Layout(width="280px"))
temp_w_controls.children[-2].children[0].on_click(temp_w_reset)
temp_w_controls.children[-2].children[1].on_click(temp_w_set_default)
temp_w_controls.children[-1].on_click(temp_w_factory)

temp_w_outputs = widgets.VBox([temp_w_status_out, temp_w_plot_out, widgets.HTML("<b>Audio</b>"), temp_w_audio_out], layout=widgets.Layout(flex="1"))
temp_widget_box = widgets.HBox([temp_w_controls, temp_w_outputs], layout=widgets.Layout(width="100%"))
with temp_w_status_out:
    temp_w_status_out.clear_output()
    if _temp_state:
        print(f"Eager tempogram: {temp_init_time:.3f}s | Tempo: {_temp_state['tempo_bpm']:.1f} BPM")
    else:
        print("Run cell above for default novelty + eager tempogram, or use Recompute.")
display(temp_widget_box)
temp_w_redraw(0)